In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/train.csv', parse_dates=[0])

In [ ]:
test = pd.read_csv('../input/test.csv', parse_dates=[0])

In [ ]:
df.rename(columns={'count': 'rentals'}, inplace=True)

In [ ]:
df['rentals'] = np.log(df['rentals'])

In [ ]:
1000 - 1100, 10 - 11

In [ ]:
import math

In [ ]:
math.log(1000) - math.log(1100), math.log(10) - math.log(11)

In [ ]:
pd.concat([df, test])

In [ ]:
df = pd.concat([df, test])

In [ ]:
df['year'] = df.datetime.dt.year
df['hour'] = df.datetime.dt.hour
df['dayofweek'] = df.datetime.dt.dayofweek

In [ ]:
test = df[df['rentals'].isnull()]
df = df[~df['rentals'].isnull()]

In [ ]:
from sklearn.model_selection import train_test_split

# Dividindo os datasets em treino e validação

In [ ]:
train, valid = train_test_split(df, random_state=42)

In [ ]:
train.count()

In [ ]:
valid.count()

In [ ]:
removed_cols = ['rentals', 'casual', 'registered', 'datetime']

In [ ]:
feats = [c for c in df.columns if c not in removed_cols]

In [ ]:
feats

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
dt = DecisionTreeRegressor(random_state=42, max_depth=2)

In [ ]:
dt.fit(train[feats],train['rentals'])

In [ ]:
from fastai.structured import draw_tree

In [ ]:
draw_tree(dt, train[feats], precision=3, size=40)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
train['preds'] = train['rentals'].mean()

In [ ]:
train.shape, train['rentals'].mean()

In [ ]:
mean_squared_error(train['rentals'], train['preds'])

In [ ]:
train[train['hour'] < 6.5]['rentals'].mean()

# MODELO FLORESTA

In [ ]:
del train['preds']

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

In [ ]:
rf.fit(train[feats], train['rentals'])

In [ ]:
train_preds = rf.predict(train[feats])

In [ ]:
train_preds

In [ ]:
mean_squared_error(train['rentals'], train_preds)**(1/2)

In [ ]:
valid_preds = rf.predict(valid[feats])

In [ ]:
mean_squared_error(valid['rentals'], valid_preds)**(1/2)

In [ ]:
test['count'] = np.exp(rf.predict(test[feats]))

In [ ]:
test[['datetime','count']].to_csv('rf.csv', index=False)